In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#loading html file as a data frame
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)


In [3]:
df=table[0] # we only need the first table
print(type(df))# checking if df is a data frame
print(df.shape)
df.head()

<class 'pandas.core.frame.DataFrame'>
(180, 3)


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


# Data cleaning
### We need to only consider the rows with an assigned Borough


In [4]:
df['Borough'].replace('Not assigned',np.NaN,inplace=True)
df.isnull().sum()

Postal code      0
Borough         77
Neighborhood    77
dtype: int64

So there are 77 rows without Boroughs, We can drop these rows

In [5]:
df.dropna(axis=0,subset=['Borough'],inplace=True)

df.head()


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


We can verify whether the drop is succesfull

In [6]:
df.isnull().sum()

Postal code     0
Borough         0
Neighborhood    0
dtype: int64

Yes the rows are correct. let us also reset the indexes

In [7]:
df=df.reset_index(drop=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [8]:
df.shape

(103, 3)

Data Frame is having 103 rows and 3 columns

# Geocoding the data
### Since Gecode is not persitent I choose to load the csv file

In [9]:
pos = pd.read_csv('https://cocl.us/Geospatial_data')
pos.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
pos.rename(columns={'Postal Code':'Postal code'},inplace=True) # making both dataframes have same column label

Now we cam use pd.merge() to join two dataframes into a new dataframe Toronto_Neighborhood

In [11]:
Toronto_Neighborhood = pd.merge(df,pos,on='Postal code')
Toronto_Neighborhood.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


We have geocoded each postal code
